
**[PT]** Português

---

**[EN]** English



# Visão geral da base de dados

---

# Database overview 


## Setup

In [1]:
from timelink.api.database import TimelinkDatabase
from timelink import version
from ucalumni.config import default_db_url

print(f"Timelink API version: {version}")
print(f"Creating TimelinkDatabase instance from {default_db_url}")
db = TimelinkDatabase(db_url=default_db_url)


Timelink API version: 1.1.14
Creating TimelinkDatabase instance from sqlite:///../database/sqlite3/fauc2.db?check_same_thread=False


In [11]:
default_db_url

'sqlite:///../database/sqlite3/fauc2.db?check_same_thread=False'

## Database status

In [3]:
from sqlalchemy.sql import func
from timelink.api.models import Entity


with db.session() as session:
    classes = session.query(Entity.pom_class,func.count(Entity.pom_class)).group_by(Entity.pom_class).all()
    for c,n in classes:
        print(f'{n:8} | {c}')


     241 | act
 2848330 | attribute
      13 | class
  155320 | person
  205436 | relation
     241 | source


## Atributos inferidos

---

## Atributes inferred

In [4]:
from sqlalchemy import func
from sqlalchemy import select
from timelink.api.models import Attribute

attr_table = Attribute.__table__

stmt = select(attr_table.c.the_type,func.count().label('tot')).group_by('the_type')
print(stmt)
print()
attributes_extracted = []
with db.session() as session:
    # nml2 = session.query(Attribute.the_type,func.count().label('tot')).group_by(Attribute.the_type).all()
    nml = session.execute(stmt)
    for the_type, tot in nml:
        print(f'{tot:6} | {the_type}')
        attributes_extracted.append((the_type,tot))

attr_not_redundand = [(t,c) for (t,c) in attributes_extracted if "." not in t]

SELECT attributes.the_type, count(*) AS tot 
FROM attributes GROUP BY attributes.the_type

   357 | colegio
105297 | código-de-referência
105297 | data-do-registo
    13 | errata
 53734 | exame
104640 | faculdade
  7071 | faculdade-original
104640 | faculdade.ano
 87110 | grau
 87110 | grau.ano
 40524 | instituta
 40524 | instituta.ano
  2445 | matricula-classe
  2445 | matricula-classe.ano
   114 | matricula-classe.obrigado
   114 | matricula-classe.obrigado.ano
    33 | matricula-classe.ordinário
    33 | matricula-classe.ordinário.ano
     2 | matricula-classe.voluntário
     2 | matricula-classe.voluntário.ano
   170 | matricula-curso
   170 | matricula-curso.ano
311844 | matricula-faculdade
311844 | matricula-faculdade.ano
  9421 | matricula-faculdade.obrigado
  9421 | matricula-faculdade.obrigado.ano
  6448 | matricula-faculdade.ordinário
  6448 | matricula-faculdade.ordinário.an
  1541 | matricula-faculdade.voluntário
  1541 | matricula-faculdade.voluntário.a
  2555 | matricula-

### Atributos inferidos (sem os atributos redundantes)

Os atributos redundantes são repetições 
de um atributos principal para facilitar 
determinadas pesquisas.

Todos os atributos com um "." no nome fornecem
uma variante do nome ou dovalor para facilitar pesquisas
e contagens. 

Por exemplo:

---

### Inferred attributes (without redundant attributes)

Redundant attributes are repetitions of an important
attribute with variants of the name or value to
facilitate retrieval or counts.

For instance:



| Atributo                     | Valor        | data 
|------------------------------|--------------|-----------
| faculdade                    | Cânones      | 1774-11-15
| faculdade.ano  	           | Cânones.1774 | 1774-11-15
| matricula-faculdade          | Matemática   |1775-00-00
| matricula-faculdade.obrigado | Matemática   | 1775-00-00



In [5]:
import pandas as pd

df = pd.DataFrame.from_records(attr_not_redundand, columns=['attr','n'])
df.sort_values('attr')

,attr,n
0,colegio,357
1,código-de-referência,105297
2,data-do-registo,105297
3,errata,13
4,exame,53734
5,faculdade,104640
6,faculdade-original,7071
7,grau,87110
8,instituta,40524
9,matricula-classe,2445


### Exemplo de registo complexo

---

### Example of complex record

Example of the current capabilities of the algorithm including correction of “Faculdade”, religious order, titles, enrollment at class level, exam results and degrees.

In [14]:
from timelink.pandas import group_attributes
from ucalumni.extractors import get_extractors
from ucalumni.aluno import get_and_process_aluno
from ucalumni.mapping import map_aluno_kperson

# id = '316297'
id = '188758'

exts = get_extractors()
print("Extractors currently active:")
[print("  ",ex.__name__) for ex in exts]
print()

print()
print("Copy the information to a markdown file it should render nicely")
print("------------------------")
print(f"https://pesquisa.auc.uc.pt/details?id={id}")
aluno = get_and_process_aluno(id,db)
print(aluno)
print("------------------------")
kaluno = map_aluno_kperson(aluno)
print("```")
print(kaluno.to_kleio())
print("```")



Extractors currently active:
   extract_name_note_vid
   extract_colegio
   extract_ordem_religiosa
   extract_naturalidade
   extract_instituta
   extract_faculdade
   extract_graus
   extract_matriculas
   extract_exames


Copy the information to a markdown file it should render nicely
------------------------
https://pesquisa.auc.uc.pt/details?id=188758
# 188758 Manuel Joaquim de Sousa Brito

## Original
```
Id: 188758
Código de referência: PT/AUC/ELU/UC-AUC/B/001-001/B/005622

Nome        : Manuel Joaquim de Sousa Brito
Data inicial: 1819-10-16
Data final  : 1845-10-22
Filiação: Manuel Joaquim de Sousa Brito
Naturalidade: Baía, Brasil
Faculdade: Leis

Matrícula(s): Filosofia 22.10.1845 (ordinário)
Direito 16.10.1819 

Exames: 3.º 12.07.1822 Aprovado Nemine Discrepante, Atos n.º 13, fl. 11v.
4.º e grau de Bacharel 14.06.1823 Aprovado Nemine Discrepante, Atos n.º 14, fl. 54v.
Formatura 14.07.1824 Aprovado Nemine Discrepante, Atos n.º 14, fl. 157v.

```
## Inferences:
* id:188758
* No

#### Mostrar os attributos como uma tabela

---

#### Show the attributes as a table

In [15]:

pd.set_option('display.max_rows',1000)
pd.set_option('display.max_colwidth',100)

df = group_attributes([id], db=db)
df.info()
df = df[~df['the_type'].str.contains(".", regex = False)]  # remove redundant attributes with "." in the name
df = df[['the_date','the_type','the_value','attr_obs']].sort_values(['the_date','the_type','the_value'])
df.fillna(" ")

<class 'pandas.core.frame.DataFrame'>
Index: 39 entries, 188758 to 188758
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   the_type   39 non-null     object
 1   the_value  39 non-null     object
 2   the_date   39 non-null     object
 3   attr_obs   39 non-null     object
dtypes: object(4)
memory usage: 1.5+ KB


,the_date,the_type,the_value,attr_obs
id,,,,
188758,1819-10-16,faculdade,(Leis),Faculdade corrigida
188758,1819-10-16,faculdade,Direito,Faculdade corrigida
188758,1819-10-16,faculdade-original,Leis,
188758,1819-10-16,matricula-faculdade,Direito,
188758,1819-10-16,naturalidade,"Baía, Brasil",
188758,1819-10-16,nome,Manuel Joaquim de Sousa Brito,
188758,1819-10-16,nome-apelido,Brito,
188758,1819-10-16,nome-apelido,Joaquim de Sousa Brito,
188758,1819-10-16,nome-apelido,Sousa Brito,
